In [1]:
import re
import json
from collections import defaultdict

In [2]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

## Read namuwiki dump

In [ ]:
with open('../data/namuwiki.json') as f:
    context_ = json.load(f)

In [ ]:
context = defaultdict(dict)
for doc in context_:
    context[doc['title']][doc['namespace']] = doc

## Namuwiki document parser

In [ ]:
regex_documnet = re.compile('\[\[(.[^:]+?)\]\]')
regex_attach = re.compile('\[\[파일:(.+?)\]\]')
regex_comment = re.compile('\[\*(.+?)\]')
regex_table = re.compile('\|\|(.+?)\|\|(.+?)\|\|')
regex_link = re.compile('\[\[(.[^:\[\]]+?)\|(.[^:\[\]]+?)\]\]')
regex_del = re.compile('~~(.+?)~~')
regex_tag = re.compile('\<(.+?)\>')

## Get drama titles from channles

In [ ]:
interested = ["JTBC 금토 드라마(2014~2017)", "JTBC 금토 드라마(2017~2020)", "JTBC 드라마", "JTBC 수목 드라마", "JTBC 월화 드라마(2011~2014)", "JTBC 월화 드라마(2017~2020)", "JTBC 주말 드라마", "KBS 수목 드라마(2001~2005)", "KBS 수목 드라마(2006~2010)", "KBS 수목 드라마(2011~2015)", "KBS 수목 드라마(2016~2020)", "KBS 월화 드라마(2001~2005)", "KBS 월화 드라마(2006~2010)", "KBS 월화 드라마(2011~2015)", "KBS 월화 드라마(2016~2020)", "KBS 학교 시리즈", "MBC 수목 미니시리즈(2006~2010)", "MBC 수목 미니시리즈(2011~2015)", "MBC 수목 미니시리즈(2016~2020)", "MBC 아침 드라마(2011~2015)", "MBC 아침 드라마(2016~2020)", "MBC 예능 드라마", "MBC 월화 미니시리즈(2006~2010)", "MBC 월화 미니시리즈(2016~2020)", "MBC 월화특별기획(2011~2015)", "MBC 일일 드라마(2016~2020)", "MBC 일일 연속극(2011~2015)", "MBC 주말 드라마(2011~2015)", "MBC 주말 드라마(2016~2020)", "MBC 주말 특별기획(2011~2015)", "MBC 주말 특별기획(2016~2020)", "MBC 하이킥 시리즈", "MBN 수목 드라마", "OCN 로맨스 드라마", "OCN 수목 오리지널", "OCN 오리지널 드라마(2010~2016)", "OCN 월화 오리지널", "OCN 토일 오리지널(2017~2020)", "SBS 금토 드라마(2019~현재)", "SBS 드라마 스페셜(1992~1995)", "SBS 드라마 스페셜(1996~2000)", "SBS 드라마 스페셜(2001~2005)", "SBS 드라마 스페셜(2006~2010)", "SBS 드라마 스페셜(2011~2015)", "SBS 드라마 스페셜(2016~2020)", "SBS 아침 연속극(2016~2020)", "SBS 월화 드라마(1991~1995)", "SBS 월화 드라마(1996~2000)", "SBS 월화 드라마(2001~2005)", "SBS 월화 드라마(2006~2010)", "SBS 월화 드라마(2011~2015)", "SBS 월화 드라마(2016~2020)", "TV CHOSUN 토일드라마", "tvN 금요 드라마(2007~2015)", "tvN 금토 드라마", "tvN 로맨스가 필요해 시리즈", "tvN 불금 시리즈(2017~)", "tvN 월화 드라마(2011~2015)", "tvN 월화 드라마(2016~2020)", "tvN 토일 드라마(2017~2020)"]

In [ ]:
titles = []

In [ ]:
for inter in interested:
    document = context[inter]
    matches = regex_documnet.findall(document['1']['text'])
    
    for match in matches:
        name, *_ = match.split('|')
        name, *_ = name.split('#')
        titles.append(name)

## Get metadata from document

In [ ]:
columns = {
    '방송 기간': ['방송기간', '방송 기간'],
    '방송 횟수': ['횟수', '방송 횟수'],
    '장르': ['장르'],
    '채널': ['채널', '방송사'],
    '제작사': ['제작사', '제작자', '제작'],
    '극본': ['극본', '대본'],
    '출연자': ['출연자', '출연', '출연진'],
}

In [ ]:
data = defaultdict(dict)

In [ ]:
notexists = []
notparser = []

In [ ]:
for title in titles:
    try:
        document = context[title]['0']['text']                       
    except KeyError:
        notexists.append(title)
        
    matches = regex_table.findall(document)
    
    d = {key: '' for key in columns}
    
    for key, value in matches:
        key = next((ckey for ckey, cvalues in columns.items() if any(cvalue in key for cvalue in cvalues)), False)
        if key:
            d[key] = value
    
    if not d:
        notparser.append(title)
    else:
        data[title] = d

## Create table from data

In [ ]:
table_columns = list(columns.keys())

In [ ]:
table = np.empty((0, len(table_columns) + 1))

In [ ]:
for title, values in data.items():
    d = [regex_tag.sub('', regex_comment.sub('', regex_del.sub(r'\1', regex_documnet.sub(r'\1', regex_link.sub(r'\1', regex_attach.sub('attach', values[column])))))).strip() 
         for column in table_columns]
    table = np.vstack((table, np.array([title, *d])))

In [ ]:
df = pd.DataFrame(table)
df.columns = ['title', *table_columns]

In [ ]:
df.head()

In [ ]:
df.to_csv('../results/namuwiki.csv', index=None)